In [1]:
import pyodbc #just installed with pip
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import pandas as pd
import warnings 

warnings.filterwarnings('ignore')

In [2]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')
 
 
# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

In [3]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary

connection_string =f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [4]:
# Now the sql query to get the data is what what you see below. 
# Note that you will not have permissions to insert delete or update this database table. 

connection = pyodbc.connect(connection_string)






In [5]:
query1 = "Select * from LP1_startup_funding2020"
data2020 = pd.read_sql(query1, connection)
 
query2 = "Select * from LP1_startup_funding2021"
data2021 = pd.read_sql(query2, connection)

In [6]:
data2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Founded        842 non-null    float64
 2   HeadQuarter    961 non-null    object 
 3   Sector         1042 non-null   object 
 4   What_it_does   1055 non-null   object 
 5   Founders       1043 non-null   object 
 6   Investor       1017 non-null   object 
 7   Amount         801 non-null    float64
 8   Stage          591 non-null    object 
 9   column10       2 non-null      object 
dtypes: float64(2), object(8)
memory usage: 82.5+ KB


In [11]:

data2020.shape

(1055, 10)

In [12]:
# Check for missing values 
data2020.isna() .any()

Company_Brand    False
Founded           True
HeadQuarter       True
Sector            True
What_it_does     False
Founders          True
Investor          True
Amount            True
Stage             True
column10          True
dtype: bool

In [9]:
data2020.isna().sum()

Company_Brand       0
Founded           213
HeadQuarter        94
Sector             13
What_it_does        0
Founders           12
Investor           38
Amount            254
Stage             464
column10         1053
dtype: int64

In [27]:
data2020_missing=data2020[data2020.isna().Founded]

In [28]:
data2020_missing

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
162,Jade Forest,NaN,Delhi,Beverages,NATURALLY THE BEST From zero artificial ingred...,"Punweet Singh, Shuchir Suri",None,250000.0,Seed,None
164,FrontRow,NaN,Bangalore,EdTech,"Comprehensive courses, taught by idols.","Mikhil Raj, Shubhadit Sharma, Ishaan Preet Singh","Lightspeed, Elevation, Deepika Padukone",3200000.0,Seed,None
180,Traya,NaN,Mumbai,HealthTech,work with our customers closely to get their h...,"Altaf Saiyed, Saloni Anand","Kae Capital, Whiteboard Capital",NaN,Seed,None
244,Hire Me Car,NaN,Noida,Car Service,India's largest cloud based digital discovery ...,Pankaj Sharma,None,NaN,Seed,None
277,Uni,NaN,Mumbai,FinTech,The team aims to bridge the gap for credit car...,Nitin Gupta,"Lightspeed India Partners, Accel India",18500000.0,Seed,None
...,...,...,...,...,...,...,...,...,...,...
1042,Super Surfaces,NaN,Hyderabad,Deisgning,A design solution based company,Kumar Varma,Vishnu Reddy,500000.0,Series A,None
1043,Quicko,NaN,Ahmedabad,Taxation,Online tax planning and filing platform,Vishvajit Sonagara,"Zerodha fintech fund, Rainmatter",280000.0,None,None
1044,Satin Creditcare,NaN,Gurgaon,Fintech,A micro finance company,None,Austrian Bank,15000000.0,None,None
1050,Leverage Edu,NaN,Delhi,Edtech,AI enabled marketplace that provides career gu...,Akshay Chaturvedi,"DSG Consumer Partners, Blume Ventures",1500000.0,None,None


In [31]:
# Check for duplicates
data2020.duplicated().sum()

3

In [33]:
#Descriptive statistics of the data .
data2020.describe()

,Founded,Amount
count,842.000000,8.010000e+02
mean,2015.363420,1.130430e+08
std,4.097909,2.476635e+09
min,1973.000000,1.270000e+04
25%,2014.000000,1.000000e+06
50%,2016.000000,3.000000e+06
75%,2018.000000,1.100000e+07
max,2020.000000,7.000000e+10


In [10]:
# Drop the specified columns
columns_to_drop = ['Stage', 'column10', 'Founded', 'Founders', 'HeadQuarter']
data2020.drop(columns_to_drop, axis=1, inplace=True)

In [11]:
data2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Sector         1042 non-null   object 
 2   What_it_does   1055 non-null   object 
 3   Investor       1017 non-null   object 
 4   Amount         801 non-null    float64
dtypes: float64(1), object(4)
memory usage: 41.3+ KB


In [20]:
data2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1206 non-null   object 
 8   Stage          781 non-null    object 
dtypes: float64(1), object(8)
memory usage: 85.1+ KB


In [21]:
data2021.shape

(1209, 9)

In [13]:
# Check for missing values 
data2021.isna() .any()

Company_Brand    False
Founded           True
HeadQuarter       True
Sector           False
What_it_does     False
Founders          True
Investor          True
Amount            True
Stage             True
dtype: bool

In [14]:
data2021.isna() .sum()

Company_Brand      0
Founded            1
HeadQuarter        1
Sector             0
What_it_does       0
Founders           4
Investor          62
Amount             3
Stage            428
dtype: int64

In [22]:
# Drop the specified columns
columns_to_drop = ['Stage','Founded', 'Founders', 'HeadQuarter']
data2021.drop(columns_to_drop, axis=1, inplace=True)

In [23]:
data2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  1209 non-null   object
 1   Sector         1209 non-null   object
 2   What_it_does   1209 non-null   object
 3   Investor       1147 non-null   object
 4   Amount         1206 non-null   object
dtypes: object(5)
memory usage: 47.4+ KB


In [16]:
data2018 = pd.read_csv('startup_funding2018.csv')
data2018.head()

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...


In [17]:
data2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company Name   526 non-null    object
 1   Industry       526 non-null    object
 2   Round/Series   526 non-null    object
 3   Amount         526 non-null    object
 4   Location       526 non-null    object
 5   About Company  526 non-null    object
dtypes: object(6)
memory usage: 24.8+ KB


In [18]:
data2018.shape

(526, 6)

In [19]:
# Check for missing values 
data2018.isna() .any()

Company Name     False
Industry         False
Round/Series     False
Amount           False
Location         False
About Company    False
dtype: bool

In [20]:
data2018.isna() .sum()

Company Name     0
Industry         0
Round/Series     0
Amount           0
Location         0
About Company    0
dtype: int64

In [21]:
data2019 = pd.read_csv('startup_funding2019.csv')
data2019.head()

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN
1,Ruangguru,2014.0,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C
2,Eduisfun,NaN,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding
3,HomeLane,2014.0,Chennai,Interior design,Provides interior designing solutions,"Srikanth Iyer, Rama Harinath","Evolvence India Fund (EIF), Pidilite Group, FJ...","$30,000,000",Series D
4,Nu Genes,2004.0,Telangana,AgriTech,"It is a seed company engaged in production, pr...",Narayana Reddy Punyala,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN


In [22]:
data2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company/Brand  89 non-null     object 
 1   Founded        60 non-null     float64
 2   HeadQuarter    70 non-null     object 
 3   Sector         84 non-null     object 
 4   What it does   89 non-null     object 
 5   Founders       86 non-null     object 
 6   Investor       89 non-null     object 
 7   Amount($)      89 non-null     object 
 8   Stage          43 non-null     object 
dtypes: float64(1), object(8)
memory usage: 6.4+ KB


In [23]:
data2019.shape

(89, 9)

In [24]:
# Check for missing values 
data2019.isna() .any()

Company/Brand    False
Founded           True
HeadQuarter       True
Sector            True
What it does     False
Founders          True
Investor         False
Amount($)        False
Stage             True
dtype: bool

In [25]:
data2019.isna() .sum()

Company/Brand     0
Founded          29
HeadQuarter      19
Sector            5
What it does      0
Founders          3
Investor          0
Amount($)         0
Stage            46
dtype: int64

In [15]:
# Drop the specified columns
columns_to_drop = ['Stage', 'Founded', 'Founders', 'HeadQuarter']
data2019.drop(columns_to_drop, axis=1, inplace=True)

In [16]:
data2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company/Brand  89 non-null     object
 1   Sector         84 non-null     object
 2   What it does   89 non-null     object
 3   Investor       89 non-null     object
 4   Amount($)      89 non-null     object
dtypes: object(5)
memory usage: 3.6+ KB
